In [1]:
# ================== 1. Setup ==================
!pip install pycocotools --quiet
import os, json, shutil, random
import torch
import torchvision
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

# ================== 2. Download COCO val2017 ==================
!wget -q http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip

# ================== 3. Select Target Classes ==================
target_classes = ['person', 'car', 'dog', 'cat', 'bicycle']
coco_annotation_file = 'annotations/instances_val2017.json'

with open(coco_annotation_file, 'r') as f:
    coco_data = json.load(f)

category_id_to_name = {cat['id']: cat['name'] for cat in coco_data['categories']}
category_name_to_id = {v: k for k, v in category_id_to_name.items()}
target_class_ids = [category_name_to_id[c] for c in target_classes]

# ================== 4. Collect Image -> Label ==================
image_id_to_label = {}
used_image_ids = set()

for ann in coco_data['annotations']:
    if ann['category_id'] in target_class_ids:
        image_id = ann['image_id']
        if image_id not in image_id_to_label:  # 1 label per image
            image_id_to_label[image_id] = ann['category_id']
            used_image_ids.add(image_id)

# ================== 5. Prepare Dataset ==================
save_dir = '/content/coco_classification'
images_dir = os.path.join(save_dir, 'images')
os.makedirs(images_dir, exist_ok=True)

image_id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}
data = []

for image_id, label_id in image_id_to_label.items():
    filename = image_id_to_filename[image_id]
    src = os.path.join('val2017', filename)
    dst = os.path.join(images_dir, filename)
    shutil.copyfile(src, dst)
    data.append((dst, target_class_ids.index(label_id)))  # class index: 0-4

# ================== 6. Split Dataset ==================
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

class COCODataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = COCODataset(train_data, transform=transform_train)
val_dataset = COCODataset(val_data, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=2)

# ================== 7. Model and Training ==================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(target_classes))
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

from torch.amp import autocast, GradScaler
scaler = GradScaler()

# ================== 8. Training Loop ==================
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * images.size(0)
    avg_train_loss = running_loss / len(train_dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
    avg_val_loss = val_loss / len(val_dataset)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# ================== 9. Save Model ==================
torch.save(model.state_dict(), '/content/drive/MyDrive/efficientnet_b0_coco5.pth')
print("✅ Model saved to Drive!")


Mounted at /content/drive


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 160MB/s]


Epoch 1, Train Loss: 0.8512, Val Loss: 0.6553
Epoch 2, Train Loss: 0.4763, Val Loss: 0.5506
Epoch 3, Train Loss: 0.3128, Val Loss: 0.5328
Epoch 4, Train Loss: 0.2148, Val Loss: 0.5460
Epoch 5, Train Loss: 0.1589, Val Loss: 0.5832
✅ Model saved to Drive!
